In [ ]:
# 690e25b4-8c5e-4a10-a32e-523da88a4c99 is my demo tenant
$TenantId = '690e25b4-8c5e-4a10-a32e-523da88a4c99' #"<TENANT_ID>"
$SubscriptionId = 'b9334351-cec8-405d-8358-51846fa2a3ab'

. "..\src\functions.ps1"

try{
    Set-AzContext -TenantId $TenantId -SubscriptionId $SubscriptionId -ErrorAction Stop
}
catch{
    Connect-AzAccount -TenantId $TenantId
}

$CheckListUrl = 'https://raw.githubusercontent.com/Azure/review-checklists/main/checklists/appsvc_security_checklist.en.json'
$appsvcSecurityChecklist = Invoke-WebRequest -Uri $CheckListUrl | Select-Object -ExpandProperty Content | ConvertFrom-Json | Select-Object -ExpandProperty items

# Subnets should have an NSG associated

Below shows subnets with no NSG associated. Delegated subnets are less important to protect using NSGs, but should still be so if at all possible.

In [ ]:
$Query = @"
resources
| where type == "microsoft.network/virtualnetworks"
| mvexpand subnets = properties.subnets
| extend Delegated = iif(array_length(subnets.properties.delegations) == 0, "no", "yes")
| where isempty(subnets.properties.networkSecurityGroup)
| project ["VNET Name"] = name, ["Subnet Name"] = subnets.name, Delegated, id = subnets.id
"@

Search-AzGraph -Query $Query | Select-Object -ExpandProperty Data | Select-Object -ExcludeProperty ResourceId | Format-Table -AutoSize

# NSG Deny All Rule

All subnets should have a deny all rule ("deny by default, permit by exception" approach). This can be implemented with a priority of 4096 (last rule evaluated always), which counters some of the default rules that ex. allows VNET to VNET (traffic tagged as VNET).

Below outputs the NSGs without such a rule.

In [ ]:
<#
powershell:
find NSGs with a "deny all" rule. as azgraph does not support antijoins we will have to find the NSGs that does NOT have such a rule
#>
$Query = @"
resources
| where type == "microsoft.network/networksecuritygroups"
| extend p = todynamic(properties.securityRules)
| mvexpand p
| where p.properties.access == "Deny" and p.properties.direction == "Inbound" and p.properties.destinationPortRange == '*' and p.properties.sourceAddressPrefix == '*'
| extend nsg_id = substring(p.id, 0, strlen(p.id) - strlen(strcat('/securityRules/',p.name)))
| join (
	resources
	| where type == "microsoft.network/networksecuritygroups"
	| project name, nsg_id = id
) on nsg_id
| project name, nsg_id
"@
[array]$NSGsWithDenyAllRule = @(Search-AzGraph -Query $Query)
# we also need all NSG IDs
$Query = @"
resources
| where type == "microsoft.network/networksecuritygroups"
| project name, nsg_id = id
"@
# $NSGs = @(
[array]$NSGs = Search-AzGraph -Query $Query
# and finally we can compare and find which NSGs does not have such a rule
Compare-Object -ReferenceObject $NSGs -DifferenceObject $NSGsWithDenyAllRule -Property nsg_id -PassThru | Select-Object name, @{name='id';expression={$_.nsg_id}}


# Open Management Ports

Below query identifies NSG rules that expose management ports to the Internet. It will not find obscure rules like allowing *3380-3390*.

Expose your Virtual Machines using Azure Bastion or other similar service. Avoid exposing VMs directly to the Internet.

In [ ]:
# check for open management ports
$Query = @"
resources
| where type == "microsoft.network/networksecuritygroups"
| extend p = todynamic(properties.securityRules)
| mvexpand p
| extend destinationPortRange = p.properties.destinationPortRange
| extend destinationPortRanges = p.properties.destinationPortRanges
| extend sourceAddressPrefix = p.properties.sourceAddressPrefix
| extend access = p.properties.access
| extend direction = p.properties.direction
| where access == "Allow" and direction == "Inbound" 
| where destinationPortRange contains "22" or destinationPortRange contains "3389" or destinationPortRanges has "22" or destinationPortRanges has "3389"
| where sourceAddressPrefix == '*' or sourceAddressPrefix == 'Internet'
| extend port = strcat(destinationPortRange, iff(array_length( destinationPortRanges) == 0, '', destinationPortRanges))
| extend nsg_id = substring(p.id, 0, strlen(p.id) - strlen(strcat('/securityRules/',p.name)))
| join kind=inner (
	resources
	| where type == "microsoft.network/networksecuritygroups"
	| project name, nsg_id = id
) on nsg_id
| project ["NSG rule name"] = p.name, ["Exposed port"] = port, ["NSG name"] = name, id = nsg_id
"@
(Search-AzGraph -Query $Query).Data | Select-Object -ExcludeProperty ResourceId